# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import time
from vaastav.fbref import *

# Getting raw data

In [3]:
# only Mo Salah
only_Salah = True

In [4]:
def get_url( url ):
    df = pd.read_html( url, header=1)[0]
    return df

In [5]:
def get_shorter_season(season):
    res = season.split("-")
    return res[0] + "-" + res[1][2:]

In [6]:
get_shorter_season("2022-2023")

'2022-23'

In [7]:
def get_logs_for_players_in_season(players, season, players_names=[]):
    logs = pd.DataFrame({'A' : []})
    for id, player in list(players.items()):
        player_name = player.data[0]['player'].replace(" ", "-").replace("ï", "i").replace("é", "e").replace("á", "a").replace("ó", "o").replace("Á", "A").replace("ø", "o").replace("ć", "c").replace("í", "i").replace("ú", "u").replace("Ć", "C").replace("ã", "a").replace("ğ", "g").replace("ş", "s").replace("É", "E").replace("ñ", "n").replace("Ł", "L").replace("ń", "n").replace("ß", "ss").replace("ç", "c").replace("İ", "I").replace("č", "c").replace("ö", "o").replace("ë", "e").replace("š", "s").replace("ä", "a").replace("Ç", "C").replace("ü", "u").replace("Ø", "O").replace("ú", "u").replace("ú", "u").replace("î", "i")
        if player_name != "Mohamed-Salah": 
            continue
        print("Getting data for " + player_name)
        url = 'https://fbref.com/en/players/' + str(id) + '/matchlogs/' + season + '/summary/' + player_name + '-Match-Logs'
        new_player_df = get_url(url)
        new_player_df["Name"] = player_name
        new_player_df["Season"] = get_shorter_season(season)
        
        
        # dropping NaN rows
        new_player_df = new_player_df[new_player_df['Date'].notna()]
        
        # only Premier League
        new_player_df = new_player_df[new_player_df.Comp == "Premier League"]
        
        # # only last 15 matches
        # new_player_df = new_player_df[-15:]
        
        if not logs.empty:
            logs = pd.concat([logs, new_player_df])
        else:
            logs = new_player_df
        
        # to not get timeouted
        time.sleep(3.2)
    return logs

In [8]:
players, stats = get_epl_players()

In [9]:
%%time
# player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
logs = get_logs_for_players_in_season(players, '2022-2023', None)

Getting data for Mohamed-Salah
CPU times: total: 78.1 ms
Wall time: 3.54 s


# Modyfing data

In [10]:
# splitting result column
logs[['WDL', 'GoalsTeams']] = logs['Result'].str.split(' ', expand=True)
logs[['Team Score', 'Opp Score']] = logs['GoalsTeams'].str.split('–', expand=True)
logs['Team Score'] = logs['Team Score'].astype(int)
logs['Opp Score'] = logs['Opp Score'].astype(int)
logs = logs.drop(['Result'], axis=1)

In [11]:
logs.replace("On matchday squad, but did not play", 0, inplace=True)
logs["Min"] = logs["Min"].astype(float)
logs["Gls"] = logs["Gls"].astype(float)
logs["Ast"] = logs["Ast"].astype(float)

In [12]:
# additional columns
# approximated FPL points
logs["CS"] = (logs["Opp Score"] == 0).astype(float)
logs["Was Home"] = (logs["Venue"] == "Home").astype(float)
logs["GW"] = logs['Round'].str.split(' ', expand=True)[1]
logs["FPL"] = 1 + (logs["Min"] >= 60) + (5 * logs["Gls"]) + (3 * logs["Ast"]) + (1 * logs["CS"])

In [13]:
logs

,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,Min,...,Name,Season,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW,FPL
1,2022-08-06,Sat,Premier League,Matchweek 1,Away,Liverpool,Fulham,Y,RW,90.0,...,Mohamed-Salah,2022-23,D,2–2,2,2,0.0,0.0,1,7.0
2,2022-08-15,Mon,Premier League,Matchweek 2,Home,Liverpool,Crystal Palace,Y,"RW,FW",90.0,...,Mohamed-Salah,2022-23,D,1–1,1,1,0.0,1.0,2,2.0
3,2022-08-22,Mon,Premier League,Matchweek 3,Away,Liverpool,Manchester Utd,Y,RW,90.0,...,Mohamed-Salah,2022-23,L,1–2,1,2,0.0,0.0,3,7.0
4,2022-08-27,Sat,Premier League,Matchweek 4,Home,Liverpool,Bournemouth,Y,RW,90.0,...,Mohamed-Salah,2022-23,W,9–0,9,0,1.0,1.0,4,3.0
5,2022-08-31,Wed,Premier League,Matchweek 5,Home,Liverpool,Newcastle Utd,Y,RW,90.0,...,Mohamed-Salah,2022-23,W,2–1,2,1,0.0,1.0,5,8.0
6,2022-09-03,Sat,Premier League,Matchweek 6,Away,Liverpool,Everton,Y,RW,90.0,...,Mohamed-Salah,2022-23,D,0–0,0,0,1.0,0.0,6,3.0
11,2022-10-01,Sat,Premier League,Matchweek 9,Home,Liverpool,Brighton,Y,"RW,RM",90.0,...,Mohamed-Salah,2022-23,D,3–3,3,3,0.0,1.0,9,5.0
13,2022-10-09,Sun,Premier League,Matchweek 10,Away,Liverpool,Arsenal,Y,RW,68.0,...,Mohamed-Salah,2022-23,L,2–3,2,3,0.0,0.0,10,2.0
15,2022-10-16,Sun,Premier League,Matchweek 11,Home,Liverpool,Manchester City,Y,FW,89.0,...,Mohamed-Salah,2022-23,W,1–0,1,0,1.0,1.0,11,8.0
16,2022-10-19,Wed,Premier League,Matchweek 12,Home,Liverpool,West Ham,Y,"RM,FW",90.0,...,Mohamed-Salah,2022-23,W,1–0,1,0,1.0,1.0,12,3.0


# Saving to csv

In [14]:
logs.to_csv("data/logs_all.csv")

# Features

In [15]:
info = ["Date", "Day", "GW", "Was Home"]
features = ["Min", "Gls", "Sh", "SoT", "xG", "npxG", "xAG", "CS"]
to_predict = ["FPL"]

In [16]:
logs[info + features + to_predict]

,Date,Day,GW,Was Home,Min,Gls,Sh,SoT,xG,npxG,xAG,CS,FPL
1,2022-08-06,Sat,1,0.0,90.0,1.0,2.0,1.0,0.4,0.4,0.1,0.0,7.0
2,2022-08-15,Mon,2,1.0,90.0,0.0,3.0,1.0,0.3,0.3,0.8,0.0,2.0
3,2022-08-22,Mon,3,0.0,90.0,1.0,3.0,1.0,0.3,0.3,0.3,0.0,7.0
4,2022-08-27,Sat,4,1.0,90.0,0.0,4.0,1.0,1.2,1.2,0.1,1.0,3.0
5,2022-08-31,Wed,5,1.0,90.0,0.0,2.0,0.0,0.2,0.2,0.4,0.0,8.0
6,2022-09-03,Sat,6,0.0,90.0,0.0,3.0,1.0,0.2,0.2,0.2,1.0,3.0
11,2022-10-01,Sat,9,1.0,90.0,0.0,5.0,2.0,0.6,0.6,0.1,0.0,5.0
13,2022-10-09,Sun,10,0.0,68.0,0.0,1.0,0.0,0.1,0.1,0.0,0.0,2.0
15,2022-10-16,Sun,11,1.0,89.0,1.0,3.0,1.0,0.9,0.9,0.0,1.0,8.0
16,2022-10-19,Wed,12,1.0,90.0,0.0,6.0,2.0,0.4,0.4,0.0,1.0,3.0


In [17]:
logs[info + features + to_predict].to_csv("data/logs_features.csv")